In [3]:
import networkx as nx
import matplotlib.pyplot as plt

In [123]:
"""
간단하게 만든 그래프, 여기서 보면 n1과 n2는 모든 node와 연결되어 있다. 
이런 경우 이 두 node는 유사하다고 평가할 수 있지 않을까? 
그렇다면 이 두가지를 matrix의 형태로 표현할 수도 있다. 
"""
g = nx.Graph()
g.add_nodes_from(
    [("n{}".format(n), {'weight':1.0}) for n in range(1, 10) ]
)
g.add_edges_from(
    [('n1', n, {'weight':1.0}) for n in g.nodes()] + [('n2', n, {'weight':1.0}) for n in g.nodes()]
)
g.remove_edge('n1', 'n2')
for n1 in g.nodes():
    try:
        g.remove_edge(n1, n1)
    except:
        continue
        

nx.draw_networkx(g, nx.shell_layout(g))
plt.axis('off')
plt.savefig('../../assets/images/markdown_img/structural_equi-201805111758.png')

"""
- 연결되어 있는지만 평가하는 matrix 
- weight를 로 평가한 matrix 

그리고 similarity를 계산한다면 어떤 distance를 활용하는 것이 적합한지도 고민이 필요함. 

bipartite한 경우는 또 다른 경우고. 

anyway, how to make matrix from graph? 
"""

import pandas as pd

node_to_node_mat = {}
for n1 in g.nodes():
    node_to_node_mat[n1] = {}
    for n2 in g.nodes():
        try:
            node_to_node_mat[n1][n2] = g[n1][n2]['weight']
        except:
            node_to_node_mat[n1][n2] = 0
node_to_node_mat = pd.DataFrame(node_to_node_mat)
print(node_to_node_mat)

     n1   n2   n3   n4   n5   n6   n7   n8   n9
n1  0.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
n2  0.0  0.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
n3  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
n4  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
n5  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
n6  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
n7  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
n8  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
n9  1.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0


In [60]:
"""
이게 유의미한지를 평가하기 위해서, 일단 필터링하지 않은 데이터지만 한번 해본다.
어떤 distance가 좋은지, 뭐 그런걸 알아야 하니까. 
"""
import pandas as pd
import itertools

excel_path_and_filename = "../../../Downloads/SMEs_Scopus_2013-2017.xlsx"
df = pd.read_excel(excel_path_and_filename)
df = df[['Author Keywords', 'Year', 'Abstract', 'Index Keywords']]

auth_kwd_lst = list(df['Author Keywords'].dropna().apply(lambda s: s.strip().split(";")))
auth_kwd_lst = map(lambda ks: [k.strip().lower() for k in ks], auth_kwd_lst)
auth_kwd_lst = list(auth_kwd_lst)
print(auth_kwd_lst[:10])

[['csfs', 'factor analysis', 'indian manufacturing smes', 'six sigma'], ['energy recovery', 'muzzle arc', 'pulsed supply', 'railgun', 'superconducting magnetic energy storage', 'system efficiency', 'termination discharge'], ['internationalization', 'market research', 'organic olive oil', 'smes', 'tourist'], ['contractors', 'entrepreneurship', 'indonesia', 'smes'], ['bpgm-sme', 'improved ukf', 'multi-target tracking', 'tracking accuracy'], ['dark triad of personality', 'entrepreneurial orientation (eo)', 'executive personality', 'small and medium-sized enterprise (sme)'], ['automated testing', 'electrical cable hardness', 'modular systems', 'small- and medium-sized enterprise (smes)', 'system adaptability', 'system automation', 'system flexibility'], ['dimension of information quality', 'quality information', 'smes'], ['discrete-event system simulation', 'productivity improvement', 'sme'], ['productivity', 'standard time', 'theory of constraint']]


In [ ]:
import collections 

auth_kwd_lst = list(df['Author Keywords'].dropna().apply(lambda s: s.strip().split(";")))
auth_kwd_lst = map(lambda ks: [k.strip().lower() for k in ks], auth_kwd_lst)
auth_kwd_lst = list(auth_kwd_lst)

auth_kwd_lst = map(lambda x: sorted(x), auth_kwd_lst)
make_edge = lambda ks: [ (ks[i], ks[j]) for i in range(0, len(ks)-1) for j in range(i+1, len(ks))]
auth_kwd_lst = itertools.chain.from_iterable(map(lambda x: make_edge(x), auth_kwd_lst))

kwdG = nx.Graph()
kwdG.add_edges_from(
    (e[0][0], e[0][1], {'weight':e[1]}) for e in collections.Counter(auth_kwd_lst).most_common()
)
print(len(kwdG.nodes()))
for e in kwdG.copy().edges(data=True):
    if e[2]['weight']<10:
        kwdG.remove_edge(e[0], e[1])
for n in kwdG.copy().nodes():
    if len(kwdG[n])==0:
        kwdG.remove_node(n)
"""
계산의 편의를 위해서 일정 weight 이하인 edge를 삭제하고 
고립된 노드 또한 삭제함 
이 네트워크를 활용해서 거리를 재보면 어떻게 나올까? 비슷한 의미를 가진 keyword는 비슷한 관계로 나올까? 
"""
print(len(kwdG.nodes()))

In [121]:
node_to_node_mat = {}
for n1 in kwdG.nodes():
    node_to_node_mat[n1] = {}
    for n2 in kwdG.nodes():
        try:
            node_to_node_mat[n1][n2] = kwdG[n1][n2]['weight']
        except:
            node_to_node_mat[n1][n2] = 0
node_to_node_mat = pd.DataFrame(node_to_node_mat)

from scipy.spatial.distance import euclidean, correlation, jaccard
"""
sparse한 관계이기 때문에 jaccard를 이용하는 것이 필요함
"""

sorted([correlation(node_to_node_mat.loc()[k1], node_to_node_mat.loc()[k2]) for k1 in node_to_node_mat.index for k2 in node_to_node_mat.index])
    

[-8.8817841970012523e-16,
 -8.8817841970012523e-16,
 -8.8817841970012523e-16,
 -8.8817841970012523e-16,
 -8.8817841970012523e-16,
 -8.8817841970012523e-16,
 -8.8817841970012523e-16,
 -8.8817841970012523e-16,
 -8.8817841970012523e-16,
 -8.8817841970012523e-16,
 -8.8817841970012523e-16,
 -8.8817841970012523e-16,
 -8.8817841970012523e-16,
 -8.8817841970012523e-16,
 -8.8817841970012523e-16,
 -8.8817841970012523e-16,
 -8.8817841970012523e-16,
 -8.8817841970012523e-16,
 -8.8817841970012523e-16,
 -8.8817841970012523e-16,
 -8.8817841970012523e-16,
 -8.8817841970012523e-16,
 -8.8817841970012523e-16,
 -8.8817841970012523e-16,
 -8.8817841970012523e-16,
 -8.8817841970012523e-16,
 -8.8817841970012523e-16,
 -8.8817841970012523e-16,
 -8.8817841970012523e-16,
 -8.8817841970012523e-16,
 -8.8817841970012523e-16,
 -8.8817841970012523e-16,
 -8.8817841970012523e-16,
 -8.8817841970012523e-16,
 -8.8817841970012523e-16,
 -8.8817841970012523e-16,
 -6.6613381477509392e-16,
 -6.6613381477509392e-16,
 -6.66133814

In [102]:
tG = nx.complete_graph(4)
tG.remove_edge(0, 1)
tG.remove_edge(0, 2)
tG.remove_edge(0, 3)
print(len(tG[0]))
tG.nodes(data=True)

0


NodeDataView({0: {}, 1: {}, 2: {}, 3: {}})